<a href="https://colab.research.google.com/github/sumaaithal/DeepLearning/blob/main/CustomerChurn_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score

## data preprocessing

In [ ]:
data = pd.read_csv("/content/Churn_Modelling.csv")

In [ ]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
X = data.iloc[:,3:-1].values
y = data.iloc[:,-1].values

In [ ]:
print(X)
print(y)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
[1 0 1 ... 1 1 0]


In [ ]:
## encoding the categorical data
### label encoding of 'gender'

In [ ]:
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])

In [ ]:
## encoding the categorical data
### one hot encoding of 'geography'

In [ ]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(),[1] )], remainder='passthrough' )
X = np.array(ct.fit_transform(X))

In [ ]:
X

array([[1.0, 0.0, 0.0, ..., 1, 1, 101348.88],
       [0.0, 0.0, 1.0, ..., 0, 1, 112542.58],
       [1.0, 0.0, 0.0, ..., 1, 0, 113931.57],
       ...,
       [1.0, 0.0, 0.0, ..., 0, 1, 42085.58],
       [0.0, 1.0, 0.0, ..., 1, 0, 92888.52],
       [1.0, 0.0, 0.0, ..., 1, 0, 38190.78]], dtype=object)

## train test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [ ]:
## feature scaling -> compulsory for deep learning

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Building ANN

In [ ]:
## initializing ANN

In [ ]:
ann = tf.keras.models.Sequential()

In [ ]:
#first hidden layer
ann.add( tf.keras.layers.Dense(units=6, activation='relu'))

#second hidden layer
ann.add( tf.keras.layers.Dense(units=6, activation='relu'))

#output layer
ann.add( tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Training & Compiling ANN

In [ ]:
#compile
ann.compile( optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
#training
ann.fit(X_train,y_train, batch_size=32, epochs=100 )

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.5986 - accuracy: 0.7110
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4860 - accuracy: 0.7950
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4517 - accuracy: 0.7994
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4363 - accuracy: 0.8035
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4266 - accuracy: 0.8083
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4200 - accuracy: 0.8183
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4149 - accuracy: 0.8246
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4107 - accuracy: 0.8300
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4060 - accuracy: 0.8336
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4010 - accura

## making predictions

In [ ]:
print(ann.predict(sc.transform([[1.0,0.0,0.0,600,1,40,3,60000,2,1,1,50000]]))>0.5)

1/1 [==============================] - 0s 30ms/step
[[False]]


In [ ]:
y_pred = ann.predict(X_test)

y_pred = (y_pred > 0.5)

print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 1ms/step
[[0 0]
 [0 0]
 [0 0]
 ...
 [1 1]
 [0 1]
 [0 1]]


In [ ]:
## confusion matrix

In [ ]:
confusion_matrix(y_test,y_pred)

array([[1530,   77],
       [ 205,  188]])

In [ ]:
accuracy_score(y_test,y_pred)

0.859